In [1]:
import pandas as pd
import datetime
import re
import requests
import bs4

# Web Scraping - Build little functions

### I. Get suppe

In [2]:
def get_suppe(url):
    resp = requests.get(url)
    return bs4.BeautifulSoup(resp.content, "html.parser")

In [3]:
url = "https://www.coursereport.com/best-coding-bootcamps"
suppe = get_suppe(url)

### II. Get list of school items

In [4]:
def get_school_list(soup):
    return (soup
            .find("ul", id="schools")
            .find_all("li"))

In [5]:
school_list = get_school_list(suppe)

In [6]:
sample = school_list[14]

### III. For each school, extract

#### 1. rank and name

In [7]:
def get_rank(school_item):
    rank_pattern = r"^(\d{1,2})\."
    rank = (school_item
            .find("h3")
            .find("a")
            .text)
    return int(re.findall(rank_pattern, rank)[0])

def get_name(school_item):
    name_pattern = r"^\d{1,2}\.\s(.+)"
    name = (school_item
            .find("h3")
            .find("a")
            .text)
    return re.findall(name_pattern, name)[0]

In [8]:
get_rank(sample)

15

In [9]:
get_name(sample)

'Ironhack'

#### 2. Get rating

In [10]:
def get_rating(school_item):
    rating_pattern = r"\((.+)\)"
    rating = (school_item
              .find("span",
                    class_="longform-rating-text")
              .text)
    
    return float(re.findall(rating_pattern, rating)[0])

In [11]:
get_rating(sample)

4.88

#### 3. Get no of reviews

In [12]:
def get_reviews(school_item):
    reviews_pattern = r"(^\d*)\s"
    reviews = (school_item
               .find_all("span",
                         class_="longform-rating-text")[1]
               .find("a")
               .text)
    return int(re.findall(reviews_pattern, reviews)[0])

In [13]:
get_reviews(sample)

903

#### 4. Get Locations

---

In [14]:
"|".join(["A", "2", "c"])

'A|2|c'

In [15]:
pd.DataFrame([{"columnA": 12,
               "columnB": "ABC",
               "columnC": "|".join(["A", "2", "c"])}])

,columnA,columnB,columnC
0,12,ABC,A|2|c


---

In [16]:
def get_locations(school_item):
    location_list = (school_item
                     .find("span",
                           class_="location")
                     .find_all("a"))
    return "|".join([loc.text for loc in location_list])

In [17]:
get_locations(sample)

'Miami|Madrid|Mexico City|Paris|Berlin|Amsterdam|Sao Paulo|Lisbon|Barcelona'

#### 5. Get description

In [18]:
def get_description(school_item):
    return (school_item
            .find("div",
                  class_="desc-container")
            .find_all("p"))[1].text

In [19]:
get_description(sample)

'Ironhack offers 9-week, full-time and 24-week, part-time bootcamps in web development, UX/UI design, and data analytics in Miami (Florida), Madrid and Barcelona (Spain), Paris (France) Mexico\xa0City (Mexico), Berlin (Germany), Amsterdam (Netherlands),\xa0Sao Paulo (Brazil) and Lisbon (Portugal).\xa0Ironhack uses a customized approach to education by allowing students to shape their experience based on personal goals. Students who graduate from the Web Development Bootcamp will be skilled in technologies like\xa0JavaScript, HTML5 and CSS3.\xa0The UX/UI program covers Design Thinking, Photoshop, Sketch, Balsamiq, InVision, and JavaScript.\xa0Data Analytics covers data wrangling/cleaning, APIs, web scraping, and intermediate topics in Git, MySQL,\xa0Python, Data visualization, Panda, and Machine Learning. And the Data Analytics program allows students to load, clean, explore and extract valuable insights from datasets and cultivate languages scuh as Python, SQL and Tableau.\xa0 '

In [20]:
def get_stars(school_item):
    stars_dict = {"icon-full_star": 1,
                  "icon-half_star": .5,
                  "icon-empty_star": 0}
    stars = (school_item
             .find("div",
                   class_="ratings title-rating")
             .find_all("span"))[1:]
    
    return float(sum([stars_dict[star["class"][0]] for star in stars]))

In [21]:
get_stars(sample)

5.0

### IV. Create a function that processes one school_item

In [22]:
def get_record(school_item):
    return {"rank": get_rank(school_item),
            "name": get_name(school_item),
            "rating": get_rating(school_item),
            "stars": get_stars(school_item),
            "reviews": get_reviews(school_item),
            "locations": get_locations(school_item),
            "description": get_description(school_item)}

In [23]:
get_record(sample)

{'rank': 15,
 'name': 'Ironhack',
 'rating': 4.88,
 'stars': 5.0,
 'reviews': 903,
 'locations': 'Miami|Madrid|Mexico City|Paris|Berlin|Amsterdam|Sao Paulo|Lisbon|Barcelona',
 'description': 'Ironhack offers 9-week, full-time and 24-week, part-time bootcamps in web development, UX/UI design, and data analytics in Miami (Florida), Madrid and Barcelona (Spain), Paris (France) Mexico\xa0City (Mexico), Berlin (Germany), Amsterdam (Netherlands),\xa0Sao Paulo (Brazil) and Lisbon (Portugal).\xa0Ironhack uses a customized approach to education by allowing students to shape their experience based on personal goals. Students who graduate from the Web Development Bootcamp will be skilled in technologies like\xa0JavaScript, HTML5 and CSS3.\xa0The UX/UI program covers Design Thinking, Photoshop, Sketch, Balsamiq, InVision, and JavaScript.\xa0Data Analytics covers data wrangling/cleaning, APIs, web scraping, and intermediate topics in Git, MySQL,\xa0Python, Data visualization, Panda, and Machine Lea

### V. Process all schools

In [24]:
school_ranking = [get_record(school) for school in school_list]

In [25]:
df = pd.DataFrame(school_ranking)

In [26]:
df

,rank,name,rating,stars,reviews,locations,description
0,1,Flatiron School,4.71,4.5,439,Brooklyn|Houston|Washington|Chicago|San Franci...,Flatiron School offers immersive on-campus and...
1,2,Hack Reactor,4.70,4.5,274,San Francisco|Online|Austin|Los Angeles|New Yo...,"Founded in 2012, Hack Reactor is a 12-week imm..."
2,3,Codesmith,4.90,5.0,358,Online|Los Angeles|New York City,"Codesmith offers a full-time, 12-week full sta..."
3,4,App Academy,4.68,4.5,826,San Francisco|Online|New York City,App Academy offers immersive web development c...
4,5,Turing,4.77,4.5,159,Denver,Turing School of Software & Design is a 7-mont...
5,6,Fullstack Academy,4.89,5.0,270,Jacksonville|San Diego|San Luis Obispo|Baton R...,Fullstack Academy offers full-time and part-ti...
6,7,General Assembly,4.32,4.5,452,Orlando|Houston|Detroit|Raleigh|London|Singapo...,General Assembly offers short and long courses...
7,8,Tech Elevator,4.95,5.0,217,Cincinnati|Pittsburgh|Philadelphia|Cleveland|C...,Tech Elevator is an immersive 14-week coding b...
8,9,DigitalCrafts,4.88,5.0,210,Kennesaw|Houston|Atlanta|Online|Seattle,DigitalCrafts offers a 16-week full-time and a...
9,10,Software Guild,4.66,4.5,146,Louisville|Atlanta|Minneapolis|Online,"The Software Guild offers immersive full-time,..."


### VI. Adjusting the functions to account for the date the data was pulled

In [27]:
print(datetime.date.today())

2020-05-13


In [28]:
def get_record(school_item, bootcamp_type, date=datetime.date.today()):
    return {"date_id": date,
            "bootcamp_type": bootcamp_type,
            "rank": get_rank(school_item),
            "name": get_name(school_item),
            "rating": get_rating(school_item),
            "stars": get_stars(school_item),
            "reviews": get_reviews(school_item),
            "locations": get_locations(school_item),
            "description": get_description(school_item)}

In [29]:
def get_rankings(school_list, bootcamp_type, date=datetime.date.today()):
    return pd.DataFrame([get_record(school, bootcamp_type, date) for school in school_list])

In [30]:
get_rankings(school_list, "coding")

,date_id,bootcamp_type,rank,name,rating,stars,reviews,locations,description
0,2020-05-13,coding,1,Flatiron School,4.71,4.5,439,Brooklyn|Houston|Washington|Chicago|San Franci...,Flatiron School offers immersive on-campus and...
1,2020-05-13,coding,2,Hack Reactor,4.70,4.5,274,San Francisco|Online|Austin|Los Angeles|New Yo...,"Founded in 2012, Hack Reactor is a 12-week imm..."
2,2020-05-13,coding,3,Codesmith,4.90,5.0,358,Online|Los Angeles|New York City,"Codesmith offers a full-time, 12-week full sta..."
3,2020-05-13,coding,4,App Academy,4.68,4.5,826,San Francisco|Online|New York City,App Academy offers immersive web development c...
4,2020-05-13,coding,5,Turing,4.77,4.5,159,Denver,Turing School of Software & Design is a 7-mont...
5,2020-05-13,coding,6,Fullstack Academy,4.89,5.0,270,Jacksonville|San Diego|San Luis Obispo|Baton R...,Fullstack Academy offers full-time and part-ti...
6,2020-05-13,coding,7,General Assembly,4.32,4.5,452,Orlando|Houston|Detroit|Raleigh|London|Singapo...,General Assembly offers short and long courses...
7,2020-05-13,coding,8,Tech Elevator,4.95,5.0,217,Cincinnati|Pittsburgh|Philadelphia|Cleveland|C...,Tech Elevator is an immersive 14-week coding b...
8,2020-05-13,coding,9,DigitalCrafts,4.88,5.0,210,Kennesaw|Houston|Atlanta|Online|Seattle,DigitalCrafts offers a 16-week full-time and a...
9,2020-05-13,coding,10,Software Guild,4.66,4.5,146,Louisville|Atlanta|Minneapolis|Online,"The Software Guild offers immersive full-time,..."


In [31]:
def get_rankings(url, bootcamp_type, date=datetime.date.today()):
    soup = get_suppe(url)
    school_list = get_school_list(soup)
    return pd.DataFrame([get_record(school, bootcamp_type, date) for school in school_list])

In [32]:
data_science_url = "https://www.coursereport.com/best-data-science-bootcamps"
get_rankings(data_science_url, "data-science")

,date_id,bootcamp_type,rank,name,rating,stars,reviews,locations,description
0,2020-05-13,data-science,1,BrainStation,4.68,4.5,352,Vancouver|Toronto|Online|New York City,BrainStation offers full-time and part-time co...
1,2020-05-13,data-science,2,Coding Temple,4.91,5.0,120,Online|Washington|Boston|Chicago|Dallas,"Coding Temple offers 10-week, full-stack codin..."
2,2020-05-13,data-science,3,Divergence Academy,5.00,5.0,20,Dallas,Divergence Academy is a 12-week full-time data...
3,2020-05-13,data-science,4,Flatiron School,4.71,4.5,439,Brooklyn|Online|Washington|Denver|Chicago|San ...,Flatiron School offers immersive on-campus and...
4,2020-05-13,data-science,5,Galvanize,4.47,4.5,177,Los Angeles|Phoenix|Denver|San Francisco|Seatt...,Galvanize offers a 13-week full-time and a 26-...
5,2020-05-13,data-science,6,General Assembly,4.32,4.5,452,Sydney|Singapore|Melbourne|London|Online|Detro...,General Assembly offers short and long courses...
6,2020-05-13,data-science,7,Metis,4.89,5.0,105,Online|Chicago|San Francisco|Seattle|New York ...,Metis offers data science training via 12-week...
7,2020-05-13,data-science,8,NYC Data Science Academy,4.84,5.0,288,Online|New York City,NYC Data Science Academy offers 12-week data s...
8,2020-05-13,data-science,9,Principal Analytics Prep,4.88,5.0,17,New York City,"Principal Analytics Prep offers a 12-week, ful..."
9,2020-05-13,data-science,10,Propulsion Academy,5.00,5.0,54,Zurich,Propulsion Academy offers 12-week full-time bo...


In [33]:
list_of_bootcamps = ["coding", "data-science", "online"]


In [34]:
def get_rankings(bootcamp_type, date=datetime.date.today()):
    url = f"https://www.coursereport.com/best-{bootcamp_type}-bootcamps"
    soup = get_suppe(url)
    school_list = get_school_list(soup)
    return pd.DataFrame([get_record(school, bootcamp_type, date) for school in school_list])

In [35]:
coding, data_science, online = [get_rankings(bc) for bc in list_of_bootcamps]

# VII. Insert data into mysql

(1) Create INSERT query

In [52]:
insert_query = "INSERT INTO coursereport.coding VALUES "
for idx, row in coding.iterrows():
    if idx != 0:
        insert_query = insert_query + ", "
        
    insert_query = insert_query + str((row["date_id"].strftime("%Y-%m-%d"),
                                       row["bootcamp_type"],
                                       row["rank"],
                                       row["name"],
                                       row["rating"],
                                       row["stars"],
                                       row["reviews"],
                                       row["locations"],
                                       row["description"]))

In [54]:
import pymysql
import getpass

In [55]:
pw = getpass.getpass()
conn = pymysql.connect(host="localhost",
                       port=3306,
                       user="ironhack",
                       passwd=pw)

········


In [56]:
cursor = conn.cursor()

In [57]:
cursor.execute(insert_query)

48

In [59]:
conn.commit()

In [ ]:
conn.close()